In [1]:
pip install opencv-python opencv-python-headless numpy


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import time

In [3]:
cap = cv2.VideoCapture(r"C:\Users\yprav\Downloads\AI Assignment video.mp4")


In [4]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [5]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))
start_time = time.time()

In [6]:
def get_quadrant(x, y, width, height):
    if x < width // 2 and y < height // 2:
        return 1
    elif x >= width // 2 and y < height // 2:
        return 2
    elif x < width // 2 and y >= height // 2:
        return 3
    else:
        return 4

In [7]:
def detect_balls(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    color_ranges = {
        'red': [(0, 70, 50), (10, 255, 255)],
        'blue': [(100, 150, 0), (140, 255, 255)],
        'green': [(40, 70, 70), (80, 255, 255)]
    }

    balls = []

    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            approx = cv2.approxPolyDP(cnt, 0.02*cv2.arcLength(cnt, True), True)
            (x, y, w, h) = cv2.boundingRect(approx)
            if w > 20 and h > 20:  # Size filter
                balls.append((x + w // 2, y + h // 2, color))

    return balls

In [8]:
event_log = []

previous_positions = {}

def log_event(ball_id, color, quadrant, event_type, timestamp):
    event_log.append(f"{timestamp:.2f}, {quadrant}, {color}, {event_type}")
    print(f"{timestamp:.2f}, {quadrant}, {color}, {event_type}")

frame_id = 0

In [9]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time() - start_time
    balls = detect_balls(frame)

    for ball in balls:
        x, y, color = ball
        quadrant = get_quadrant(x, y, width, height)
        ball_id = f"{color}_{x}_{y}"

        if ball_id not in previous_positions:
            log_event(ball_id, color, quadrant, "Entry", current_time)
        else:
            prev_x, prev_y = previous_positions[ball_id]
            prev_quadrant = get_quadrant(prev_x, prev_y, width, height)
            if prev_quadrant != quadrant:
                log_event(ball_id, color, prev_quadrant, "Exit", current_time)
                log_event(ball_id, color, quadrant, "Entry", current_time)

        previous_positions[ball_id] = (x, y)

    cv2.line(frame, (width // 2, 0), (width // 2, height), (255, 255, 255), 2)
    cv2.line(frame, (0, height // 2), (width, height // 2), (255, 255, 255), 2)

    for ball in balls:
        x, y, color = ball
        cv2.circle(frame, (x, y), 10, (0, 0, 255), -1)  # Example color

    out.write(frame)
    frame_id += 1

cap.release()
out.release()

with open('events.txt', 'w') as f:
    for event in event_log:
        f.write(event + '\n')

0.08, 2, red, Entry
0.08, 2, red, Entry
0.08, 1, red, Entry
0.08, 4, blue, Entry
0.08, 2, blue, Entry
0.08, 2, blue, Entry
0.12, 3, red, Entry
0.12, 1, red, Entry
0.12, 1, red, Entry
0.12, 4, blue, Entry
0.12, 2, blue, Entry
0.15, 3, red, Entry
0.15, 4, red, Entry
0.15, 3, red, Entry
0.15, 2, red, Entry
0.19, 3, red, Entry
0.19, 4, red, Entry
0.19, 4, red, Entry
0.19, 3, red, Entry
0.19, 2, red, Entry
0.19, 1, red, Entry
0.23, 4, red, Entry
0.23, 2, red, Entry
0.23, 2, red, Entry
0.23, 2, blue, Entry
0.27, 4, red, Entry
0.27, 3, red, Entry
0.27, 2, red, Entry
0.27, 2, blue, Entry
0.31, 3, red, Entry
0.31, 4, red, Entry
0.31, 2, red, Entry
0.31, 1, red, Entry
0.35, 3, red, Entry
0.38, 3, red, Entry
0.38, 1, red, Entry
0.38, 2, blue, Entry
0.42, 3, red, Entry
0.42, 3, red, Entry
0.42, 2, red, Entry
0.42, 1, red, Entry
0.42, 2, blue, Entry
0.46, 3, red, Entry
0.46, 3, red, Entry
0.46, 4, red, Entry
0.46, 3, red, Entry
0.46, 2, red, Entry
0.46, 2, blue, Entry
0.50, 3, red, Entry
0.50, 3, r